In [1]:
!pip install -q kaggle

In [2]:
import numpy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import classification_report
from google.colab import files
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

In [3]:
 # Preparando para usar API do kaggle

 files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"pedrojeronimo","key":"e01e602764cdd6488501411a5c2321e4"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
# Baixando dataset diretamente do kaggle

!kaggle datasets download -d danielefm/pesquisa-de-mobilidade-urbana-do-distrito-federal

  0% 0.00/4.37M [00:00<?, ?B/s]
100% 4.37M/4.37M [00:00<00:00, 217MB/s]


In [ ]:
# Descompactando arquivo zip contendo o dataset

!unzip pesquisa-de-mobilidade-urbana-do-distrito-federal.zip
!rm pesquisa-de-mobilidade-urbana-do-distrito-federal.zip
!ls

In [7]:
data = pd.read_csv("/content/Domicilio.csv", sep=';')
print(data.head())

   domicilio_id  numero_residentes  banheiros  quartos  veiculos  bicicletas  \
0             3                  2          2        2         1           0   
1             4                  4          1        3         3           0   
2             7                  3          2        3         1           0   
3            22                  3          4        4         2           0   
4            35                  3          2        3         1           1   

   motos        tipo_imovel                          renda condicao_moradia  \
0      0  Casa de Alvenaria   De R$ 4.400,00 a R$ 8.800,00          Própria   
1      0  Casa de Alvenaria   De R$ 1.760,00 a R$ 2.640,00          Própria   
2      0  Casa de Alvenaria   De R$ 2.640,00 a R$ 4.400,00          Própria   
3      0  Casa de Alvenaria   De R$ 4.400,00 a R$ 8.800,00          Própria   
4      0        Apartamento  De R$ 8.800,00 a R$ 13.200,00          Própria   

                  agua_encanada         rua_

In [8]:
# Checando colunas com nulos

data.isnull().sum()

domicilio_id                      0
numero_residentes                 0
banheiros                         0
quartos                           0
veiculos                          0
bicicletas                        0
motos                             0
tipo_imovel                       0
renda                             0
condicao_moradia                  0
agua_encanada                     0
rua_pavimentada                   0
vaga_propria_estacionamento       0
ano_veiculo_mais_recente       6315
empregados_domesticos             0
tv_cabo                           0
recebe_bolsa_familia            128
fexp_domicilio                    0
macrozona                         0
regiao_administrativa             0
dtype: int64

In [9]:
# Removendo valores nulos nas colunas escolhidas e que possuiam valores nulos

data = data.dropna(subset=['recebe_bolsa_familia'])

In [10]:
data2 = data[["banheiros", "quartos", "veiculos", "bicicletas",
              "motos", "tipo_imovel", "renda", "condicao_moradia",
              "agua_encanada", "rua_pavimentada", "recebe_bolsa_familia","numero_residentes"]].copy()

In [11]:
# Checando os valores que cada coluna pode receber

for coluna in data2.columns:
  print(data2[coluna].value_counts())

1      9004
2      6059
3      2377
4       819
5       438
6       204
7       116
8        51
9        27
0        11
11        6
12        5
10        4
105       1
31        1
303       1
Name: banheiros, dtype: int64
3      6811
2      6377
1      2856
4      2171
5       586
6       195
7        59
0        19
8        15
20        7
10        7
30        4
9         4
31        3
21        3
12        2
23        1
311       1
52        1
16        1
11        1
Name: quartos, dtype: int64
1     8080
0     6289
2     3556
3      882
4      245
5       55
6       10
10       3
7        3
8        1
Name: veiculos, dtype: int64
0     11550
1      4417
2      2139
3       693
4       243
5        57
6        13
7         5
8         4
9         2
11        1
Name: bicicletas, dtype: int64
0     17512
1      1506
2        90
3        11
9         2
6         1
20        1
4         1
Name: motos, dtype: int64
Casa de Alvenaria              13861
Apartamento                     4781


In [12]:
# Tratando dados categóricos que estão como string para número:

# tipo_imovel

imoveis = {
    "Casa de Alvenaria": 0,
    "Apartamento": 1,
    "Alvenaria Inacabada": 2,
    "Prédio Comercial/Industrial": 3,
    "Barraco (com piso)": 4,
    "Barraco (sem piso)": 5,
    "Outros": 6
}

data2['tipo_imovel'] = data2['tipo_imovel'].map(imoveis)


# renda

faixas_de_renda = {
    "Até R$ 440,00":0,
    "De R$ 440,00 a R$ 880,00":1,
    "De R$ 880,00 a R$ 1.760,00":2,
    "De R$ 1.760,00 a R$ 2.640,00":3,
    "De R$ 2.640,00 a R$ 4.400,00":4,
    "De R$ 4.400,00 a R$ 8.800,00":5,
    "De R$ 8.800,00 a R$ 13.200,00":6,
    "De R$ 13.200,00 a R$ 17.600,00":7,
    "De R$ 17.600,00 a R$ 22.000,00":8,
    "De R$ 22.000,00 a R$ 26.400,00":9,
    "Acima de R$ 26.400,00":10,
    "Sem Renda":11,
    "Não respondeu":12
}


data2['renda'] = data2['renda'].map(faixas_de_renda)

# condicao_moradia

tipo_residencia = {
    "Própria": 0,
    "Alugada": 1,
    "Cedida": 2,
    "Funcional": 3,
    "Própria em Aquisição": 4,
    "Concessão de Uso": 5,
    "Outros": 6,
    "Não Respondeu": 7
}

data2['condicao_moradia'] = data2['condicao_moradia'].map(tipo_residencia)

# agua_encanada

agua_encanada = {
    "Sim, em pelo menos um cômodo": 0,
    "Sim, só na propriedade ou terreno": 1,
    "Não": 2
}
data2['agua_encanada'] = data2['agua_encanada'].map(agua_encanada)

# rua_pavimentada

rua_pavimentada = {
    "Asfaltada, pavimentada": 0,
    "Terra, cascalho": 1
}

data2['rua_pavimentada'] = data2['rua_pavimentada'].map(rua_pavimentada)

#recebe_bolsa_familia

recebe_bolsa_familia = {
    "Não": 0,
    "Sim": 1
}

data2['recebe_bolsa_familia'] = data2['recebe_bolsa_familia'].map(recebe_bolsa_familia)


In [13]:
# Filtrando para apenas duas classes

data2 = data2[data2['tipo_imovel'] <= 1]

In [14]:
# Checar balanceamento das classes

print(data2['tipo_imovel'].value_counts())

0    13861
1     4781
Name: tipo_imovel, dtype: int64


In [15]:
df_x = data2[["banheiros", "quartos", "veiculos", "bicicletas", "motos", "numero_residentes", "renda", "condicao_moradia", "agua_encanada", "rua_pavimentada", "recebe_bolsa_familia"]].copy()
df_y = data2[['tipo_imovel']].copy()

In [16]:
# Checando nulos e o formato do dataset X e y

print(f'''
X:
{df_x.isnull().sum()}
dimensões de X:
{df_x.shape}
''')

print(f'''
y:
{df_y.isnull().sum()}
dimensões de y:
{df_y.shape}
''')


X:
banheiros               0
quartos                 0
veiculos                0
bicicletas              0
motos                   0
numero_residentes       0
renda                   0
condicao_moradia        0
agua_encanada           0
rua_pavimentada         0
recebe_bolsa_familia    0
dtype: int64
dimensões de X:
(18642, 11)


y:
tipo_imovel    0
dtype: int64
dimensões de y:
(18642, 1)



In [17]:
df_x.head()

,banheiros,quartos,veiculos,bicicletas,motos,numero_residentes,renda,condicao_moradia,agua_encanada,rua_pavimentada,recebe_bolsa_familia
0,2,2,1,0,0,2,5,0,0,0,0
1,1,3,3,0,0,4,3,0,0,0,0
2,2,3,1,0,0,3,4,0,0,0,0
3,4,4,2,0,0,3,5,0,0,0,0
4,2,3,1,1,0,3,6,0,0,0,0


In [18]:
df_y.head()

,tipo_imovel
0,0
1,0
2,0
3,0
4,1


In [19]:
X = np.array(df_x)
y = np.array(df_y)

In [20]:
# Cria uma instância do RandomUnderSampler
rus = RandomUnderSampler(random_state=0)

# Realiza o undersampling
X_balanced, y_balanced = rus.fit_resample(X, y)

In [21]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_balanced, y_balanced, test_size=0.10, random_state=42)

In [22]:
print(Counter([i for i in ytrain.tolist()]))

Counter({0: 4305, 1: 4300})


In [23]:
# Instanciando modelos

knearestclassifier = KNeighborsClassifier()
decisiontree = DecisionTreeClassifier()
logisticregression = LogisticRegression(max_iter=1000)
passiveAggressive = PassiveAggressiveClassifier()
svm_classifier = SVC()
bernoulli_naiveBayes = BernoulliNB()


In [24]:
# Treinando modelos

knearestclassifier.fit(Xtrain, ytrain)
decisiontree.fit(Xtrain, ytrain)
logisticregression.fit(Xtrain, ytrain)
passiveAggressive.fit(Xtrain, ytrain)
svm_classifier.fit(Xtrain, ytrain)
bernoulli_naiveBayes.fit(Xtrain, ytrain)

BernoulliNB()

In [25]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculando as previsões para o conjunto de teste
knn_preds = knearestclassifier.predict(Xtest)
dt_preds = decisiontree.predict(Xtest)
lr_preds = logisticregression.predict(Xtest)
pa_preds = passiveAggressive.predict(Xtest)
svm_preds = svm_classifier.predict(Xtest)
bnb_preds = bernoulli_naiveBayes.predict(Xtest)

# Criando listas para armazenar as métricas para cada modelo
precision_list = []
recall_list = []
f1_list = []

# Calculando as métricas para cada modelo e adicionando às listas
for preds in [knn_preds, dt_preds, lr_preds, pa_preds, svm_preds, bnb_preds]:
    precision_list.append(precision_score(ytest, preds, average='macro'))
    recall_list.append(recall_score(ytest, preds, average='macro'))
    f1_list.append(f1_score(ytest, preds, average='macro'))

# Adicionando as métricas ao dicionário
data3 = {
    "Classification Algorithms": ["KNN Classifier", "Decision Tree Classifier",
                                  "Logistic Regression", "Passive Aggressive Classifier",
                                  "SVM Classifier", 'Bernoulli Naive Bayes'],
    "Score": [knearestclassifier.score(Xtest, ytest), decisiontree.score(Xtest, ytest),
              logisticregression.score(Xtest, ytest), passiveAggressive.score(Xtest, ytest),
              svm_classifier.score(Xtest, ytest), bernoulli_naiveBayes.score(Xtest, ytest)],
    "Precision": precision_list,
    "Recall": recall_list,
    "F1 Score": f1_list
}

In [26]:
# Rankeando os modelos

score = pd.DataFrame(data3)
score = score.sort_values(by='Score', ascending=False); score


,Classification Algorithms,Score,Precision,Recall,F1 Score
4,SVM Classifier,0.766980,0.766975,0.766968,0.766971
0,KNN Classifier,0.739812,0.740565,0.739657,0.739521
1,Decision Tree Classifier,0.725183,0.725541,0.725279,0.725124
2,Logistic Regression,0.710554,0.710690,0.710475,0.710451
3,Passive Aggressive Classifier,0.655172,0.721368,0.653733,0.626066
5,Bernoulli Naive Bayes,0.613375,0.651630,0.612043,0.585782
